## CPSC 340 Lecture 22: stochastic gradient descent

This notebook is for the in-class activities. It assumes you have already watched the [associated video](https://www.youtube.com/watch?v=lmqV5Z5HZzc&list=PLWmXHcz_53Q02ZLeAxigki1JZFfCO6M-b&index=22).

<font color='red'>**REMINDER TO START RECORDING**</font>

Also, reminder to enable screen sharing for Participants.

## Pre-class music



1. 911 by Lady Gaga
2. 夜に駆ける by YOASOBI

## Admin

- a5 posted

## Video chapters

- kernel trick recap
- stochastic gradient motivation
- SGD vs. GD
- minimizing averages
- per-example gradients
- visualizing SGD: 1 parameter
- decreasing step sizes
- stochastic average gradient
- visualizing SGD: 2 parameters
- summary